In [ ]:
import numpy as np

class Activation:
    
    def relu(self, x):
        return np.maximum(x, 0)
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    def tanh(self, x):
        return np.tanh(x)
    def linear(self, x):
        return x
    def softmax(self, x):
        exps = np.exp(x - x.max())
        return exps / np.sum(exps)

    def g_relu(self, x):
        return 1 * (x > 0)
    def g_sigmoid(self, x):
        return (1 - x) * x
    def g_tanh(self, x):
        return 1 - x*x
    def g_linear(self, x):
        return 1 * (x==x)
    def g_softmax(self, x):
        dx_ds = np.diag(x) - np.dot(x, x.T)
        return dx_ds.sum(axis=0).reshape(-1, 1) 

    
    def __init__(self, acti):
        funcs = {
            "TANH" : self.tanh,
            "SIGMOID" : self.sigmoid,
            "RELU" : self.relu,
            "LINEAR" : self.linear,
            "SOFTMAX" : self.softmax
        }

        grads = {
            "TANH" : self.g_tanh,
            "SIGMOID" : self.g_sigmoid,
            "RELU" : self.g_relu,
            "LINEAR" : self.g_linear,
            "SOFTMAX" : self.g_softmax
        }
        self.acti = acti
        self.func = funcs[acti]
        self.grad = grads[acti]
        
        return       
    
    def __str__(self):
        s = "\nActivation:" + self.acti
        return s

In [ ]:
import random

class LinearLayer:

    def __init__(self, n_in, n_node, W=None, b=None):
        self.n_node = n_node
        self.n_in = n_in
        
        size = n_in*n_node
        if W == None:
            weights = np.random.uniform(-1, 1, size=size).reshape(n_node, n_in)
            scale = np.sqrt(2./size)
            self.W = weights * scale
        else: 
            nu, ni = W.shape
            if nu != n_node or ni != n_in:
                raise ValueError("Incorrect weights input")
            else:
                self.W = W
              
        self.b = b if b != None else np.zeros(shape=(n_node, 1))

        #self.W = np.ones(shape=(n_node, n_in))
        #self.b = np.ones(shape=(n_node, 1))

        self.x = None
        self.signal = None
        self.y = None

        self.in_grad = None
        self.G = None
        self.g_b = None
        self.out_grad = None
        return
        
    def forward(self, x):
        self.x = x
        self.y = np.dot(self.W, x) + self.b
        self.signal = self.y
        return self.y

    def backward(self, node_grad):       
        self.G = np.outer(node_grad, self.x)
        self.g_b = node_grad
        self.out_grad = np.dot(self.W.T, node_grad)         
        return self.out_grad

    def update(self, learning_rate):
        self.W = self.W - self.G * learning_rate
        self.b = self.b - self.g_b * learning_rate
        return
            
    def __str__(self):
        s = "\nx is:\n"+str(self.x)
        s += "\nW is:\n" + str(self.W)
        s += "\nb is:\n" + str(self.b)
        s += "\ny is:\n"+str(self.y)
        s += "\nin_grad is:\n"+str(self.in_grad)
        s += "\nG is:\n"+str(self.G)
        s += "\ng_b is:\n"+str(self.g_b)
        s += "\nout_grad is:\n"+str(self.out_grad)
        return s
        

In [ ]:
class PercepLayer(LinearLayer):
        
    def __init__(self, n_in, n_node, W=None, b=None, acti="RELU"):
        super().__init__(n_in, n_node, W=None, b=None)
        self.activation = Activation(acti)
        return
        
    def forward(self, x):
        if x.size != self.n_in: 
            raise ValueError("Incorrect data input", x.size, self.n_in)
        # linear part first
        signal = super().forward(x)
        # nonlinear
        self.y = self.activation.func(signal) 
        return self.y
      
    def backward(self, in_grad):
        if in_grad.size != self.n_node: 
            raise ValueError("Incorrect data input")
        # nonlinear    
        self.in_grad = in_grad
        node_grad = self.activation.grad(self.y)
        node_grad = node_grad * in_grad
        # linear
        self.out_grad = super().backward(node_grad)
        return self.out_grad

    def update(self, learning_rate):
        super().update(learning_rate)
        return

    def __str__(self):
        s = super().__str__()
        s += str(self.activation)
        return s
            

In [ ]:
class SoftMaxLayer(PercepLayer):
        
    def __init__(self, n_in, n_node, W=None, b=None):
        super().__init__(n_in, n_node, W=None, b=None, acti="SOFTMAX")
        self.predict = None
        self.truth = None
        return
        
    def forward(self, x):
        # hidden part first, computing softmax
        self.y = super().forward(x)
        self.predict = self.y.argmax()
        return self.predict
      
    def backward(self, in_truth):
        # softmax gradient, dL/ds (combining the cost and activation layer)
        self.truth = np.zeros(shape=(self.n_node,1))
        self.truth[in_truth] = 1
        node_grad = self.y - self.truth
        # then linear layer
        self.out_grad = LinearLayer.backward(self, node_grad)
        return self.out_grad
    
    def update(self, learning_rate):
        super().update(learning_rate)
        return
    
    def __str__(self):
        s = super().__str__()
        s += "\nPrediction:\n" + str(self.predict)
        s += "\nTruth:\n" + str(self.truth)
        return s
            

In [ ]:
class MLP:
        
    def __init__(self, n_in=28*28, n_node=28*28+1, n_class=10, learning_rate=0.1):   

        self.type = "MLP"

        self.n_in = n_in
        self.n_node = n_node if (n_node != None) else (n_in+ 1)
        self.hidden_layer1 = PercepLayer(n_in, n_node)
        #self.hidden_layer2 = PercepLayer(n_node, n_node)
        self.output_layer = SoftMaxLayer(n_node, n_class)
        
        self.learning_rate = learning_rate
        return
    
    def forward(self, x):
        y = self.hidden_layer1.forward(x)
        #y = self.hidden_layer2.forward(y)
        self.output_layer.forward(y)
        return
    
    def backward(self, label):
        out_grad = self.output_layer.backward(label)
        #out_grad = self.hidden_layer2.backward(out_grad)
        self.hidden_layer1.backward(out_grad)
        return
    
    def update(self):
        self.hidden_layer1.update(self.learning_rate)
        #self.hidden_layer2.update(self.learning_rate)
        self.output_layer.update(self.learning_rate)
        return
        
    def train_1sample(self, x, label):
        self.forward(x)
        self.backward(label)
        self.update()
        return
    
    def predict_1sample(self, x):
        y = self.hidden_layer1.forward(x)
        #y = self.hidden_layer2.forward(y)
        predict = self.output_layer.forward(y)
        return predict
 

In [ ]:
import gzip
import struct

import matplotlib as mpl
import matplotlib.pyplot as plt

# pre-requirement: MNIST files from http://yann.lecun.com/exdb/mnist/ stored in local directory
class MnistInput:
    def __init__(self, data):
        if data == "training":
            zX = './train-images-idx3-ubyte.gz'
            zy = './train-labels-idx1-ubyte.gz'
        elif data == "testing":
            zX = './t10k-images-idx3-ubyte.gz'
            zy = './t10k-labels-idx1-ubyte.gz'
        else: raise ValueError("Incorrect data input")
        
        self.zX = zX
        self.zy = zy
        return
    
    def read(self, num):

        zX = self.zX
        zy = self.zy
        with gzip.open(zX) as fX, gzip.open(zy) as fy:
            magic, nX, rows, cols = struct.unpack(">IIII", fX.read(16))
            magic, ny = struct.unpack(">II", fy.read(8))
            if nX != ny: raise ValueError("Inconsistent data and label files")

            img_size = cols*rows
            if num <= 0 or num > nX: num = nX 
            for i in range(num):
                X = struct.unpack("B"*img_size, fX.read(img_size))
                X = np.array(X).reshape(rows, cols)
                y, = struct.unpack("B", fy.read(1))
                yield (X, y)
        return
    


In [4]:
from IPython.core.debugger import set_trace
class MNIST:

    def __init__(self, nn):   
        self.nn = nn
        self.train_input = MnistInput("training")
        self.test_input = MnistInput("testing")
        return
    
    def train(self, n_sample):
        i = 1
        for X, y in self.train_input.read(n_sample):
            X = X/255
            if self.nn.type == "MLP":
                X = X.reshape(-1,1)                
            else:
                X = np.array([X])
            #print("Training: ", i); i+=1
            #if i==100 or i==200: set_trace()
            self.nn.train_1sample(X, y)
        return
            
    def test(self, n_sample):
        correct = 0
        total = 0
        for X, y in self.test_input.read(n_sample):
            aX = X/255
            if self.nn.type == "MLP":
                aX = aX.reshape(-1,1)
            else:
                aX = np.array([aX])

            predict = self.nn.predict_1sample(aX)
            correct += 1 * (predict == y)
            total += 1
            #print("\nPredict {} to be {}:".format(y, predict))
            #plt.imshow(X, cmap=mpl.cm.Greys)
            #plt.show()
            
        accuracy = correct/total
        return accuracy

In [ ]:
def run_mlp_mnist():
    mlp = MLP()
    mnist = MNIST(mlp)
    mnist.train(-1)
    accuracy = mnist.test(-1)
    print("\nAccuracy: ", accuracy)


In [ ]:
def run_mlp_test():
    mlp = MLP(3,4,3)
    X = [
        [1,2,3],
        [2,1,3],
        [3,1,2],
        [3,2,1],
        [1,3,2],
        [2,3,1]
        ]
    Y = [2,2,0,0,1,1]
    X = np.array(X)
    Y = np.array(Y)

    for j in range(40):
        for i in range(X.shape[0]):
            mlp.train_1sample(X[i].reshape(-1,1), Y[i])

    X = np.array([[1,2,3],[2,3,1],[3,1,2]])
    for i in range(X.shape[0]):
        predict = mlp.predict_1sample(X[i].reshape(-1,1))
        print("\nPredict: ", X[i], predict)


In [ ]:
def is_main_module():
    return __name__ == '__main__' and '__file__' not in globals()

if is_main_module():
    #run_mlp_test()
    run_mlp_mnist()